<a href="https://colab.research.google.com/github/mittushaji25/crypto-xrp-analysis/blob/main/xrp_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧠 Project Step 1: Data Extraction

This step extracts historical XRPUSDT trading data from Binance using their public API. The data includes timestamped information and is saved as a raw CSV file for further cleaning and analysis.

#### ✅ Outputs:
- `xrpusdt_combined.csv`

#### 🔍 Key Tasks:
- Connect to Binance API
- Fetch monthly data starting from January 2023 to till date
- Combine data from multiple files to one for reproducibility


In [ ]:
import requests, zipfile, io
import pandas as pd
from datetime import datetime

In [ ]:
years = [2023, 2024, 2025]
months = [f"{i:02d}" for i in range(1, 13)]

all_data = pd.DataFrame()

for year in years:
    for month in months:
        # Skip future months in 2025
        if year == 2025 and int(month) > datetime.now().month:
            continue

        url = f"https://data.binance.vision/data/spot/monthly/klines/XRPUSDT/1d/XRPUSDT-1d-{year}-{month}.zip"
        try:
            response = requests.get(url)
            response.raise_for_status()

            with zipfile.ZipFile(io.BytesIO(response.content)) as z:
                z.extractall("xrpusdt_data")
                csv_name = z.namelist()[0]

            df = pd.read_csv(f"xrpusdt_data/{csv_name}", header=None)
            df.columns = [
                "Open Time", "Open", "High", "Low", "Close", "Volume",
                "Close Time", "Quote Asset Volume", "Number of Trades",
                "Taker Buy Base Volume", "Taker Buy Quote Volume", "Ignore"
            ]

            all_data = pd.concat([all_data, df], ignore_index=True)
        except Exception as e:
            print(f"❌ Failed for {year}-{month}: {e}")


❌ Failed for 2025-07: 404 Client Error: Not Found for url: https://data.binance.vision/data/spot/monthly/klines/XRPUSDT/1d/XRPUSDT-1d-2025-07.zip


In [ ]:
all_data.to_csv("xrpusdt_combined.csv", index=False)

# 🧪 Project Step 2: Data Exploration

Explore the raw XRPUSDT dataset to understand its structure, identify potential issues, and guide the next steps in cleaning and transformation.

#### 🔍 Key Questions:
- What columns are present, and what do they represent?
- Are there missing values or duplicates?
- Are timestamps valid and consistent?
- Are there outliers or anomalies in price or volume?
- What is the overall shape and distribution of the data?

In [ ]:
all_data.shape
all_data.columns
all_data.head()

,Open Time,Open,High,Low,Close,Volume,Close Time,Quote Asset Volume,Number of Trades,Taker Buy Base Volume,Taker Buy Quote Volume,Ignore
0,1672531200000,0.3389,0.3400,0.3354,0.3387,102026821.0,1672617599999,3.447164e+07,48661,53592277.0,1.810790e+07,0
1,1672617600000,0.3386,0.3555,0.3000,0.3482,647520487.0,1672703999999,2.178040e+08,285660,320823575.0,1.080569e+08,0
2,1672704000000,0.3482,0.3520,0.3395,0.3436,346410334.0,1672790399999,1.195113e+08,154113,172318472.0,5.944846e+07,0
3,1672790400000,0.3436,0.3605,0.3390,0.3472,372672796.0,1672876799999,1.295853e+08,147677,193027855.0,6.713928e+07,0
4,1672876800000,0.3471,0.3487,0.3333,0.3380,265529668.0,1672963199999,9.073585e+07,106910,129294274.0,4.418781e+07,0


In [ ]:
# Check data types
all_data.dtypes

,0
Open Time,int64
Open,float64
High,float64
Low,float64
Close,float64
Volume,float64
Close Time,int64
Quote Asset Volume,float64
Number of Trades,int64
Taker Buy Base Volume,float64


In [ ]:
# Summary statistics
all_data.describe()

,Open Time,Open,High,Low,Close,Volume,Close Time,Quote Asset Volume,Number of Trades,Taker Buy Base Volume,Taker Buy Quote Volume,Ignore
count,9.120000e+02,912.000000,912.000000,912.000000,912.000000,9.120000e+02,9.120000e+02,9.120000e+02,9.120000e+02,9.120000e+02,9.120000e+02,912.0
mean,3.473826e+14,0.970641,1.003148,0.936285,0.972723,3.700959e+08,3.473998e+14,3.492578e+08,7.728642e+05,1.833062e+08,1.726876e+08,0.0
std,6.950753e+14,0.792662,0.824639,0.757598,0.793491,2.941272e+08,6.951098e+14,5.077632e+08,1.211049e+06,1.468311e+08,2.526205e+08,0.0
min,1.672531e+12,0.338000,0.340000,0.300000,0.338000,4.052670e+07,1.672618e+12,2.750721e+07,3.836700e+04,1.998583e+07,1.261434e+07,0.0
25%,1.692209e+12,0.502775,0.511175,0.488975,0.502950,1.887051e+08,1.692295e+12,1.304329e+08,1.558885e+05,9.157859e+07,6.367682e+07,0.0
50%,1.711886e+12,0.561300,0.576600,0.543200,0.562050,3.031664e+08,1.711973e+12,1.949285e+08,2.870230e+05,1.521314e+08,9.759324e+07,0.0
75%,1.731564e+12,0.816275,0.930425,0.770550,0.837975,4.425727e+08,1.731650e+12,3.478961e+08,8.052872e+05,2.201800e+08,1.704561e+08,0.0
max,1.751242e+15,3.292300,3.400000,3.175800,3.292200,2.543717e+09,1.751328e+15,6.398461e+09,1.149188e+07,1.284592e+09,3.232098e+09,0.0


In [ ]:
# Missing values
all_data.isnull().sum()

,0
Open Time,0
Open,0
High,0
Low,0
Close,0
Volume,0
Close Time,0
Quote Asset Volume,0
Number of Trades,0
Taker Buy Base Volume,0


In [ ]:
# Duplicates
all_data.duplicated().sum()

np.int64(0)

In [ ]:
# Timestamps coversion from Unix -- Open & Close Time
all_data['Open Time'] = pd.to_datetime(all_data['Open Time'], unit='ms', errors='coerce')
all_data['Close Time'] = pd.to_datetime(all_data['Close Time'], unit='ms', errors='coerce')

,Open Time
0,2023-01-01
1,2023-01-02
2,2023-01-03
3,2023-01-04
4,2023-01-05
...,...
907,NaT
908,NaT
909,NaT
910,NaT
